# Topic Modeling with gensim
We'll try out [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) in [gensim](http://radimrehurek.com/gensim/index.html) on the [20 Newsgroups dataset](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) with some simple preprocessing.

#### Install gensim

In [3]:
# !conda install gensim -y

##### imports

In [4]:
# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Let's retain only a subset of the 20 categories in the original 20 Newsgroups Dataset.

In [5]:
# Set categories
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball', 
              'rec.motorcycles', 'sci.space', 'talk.politics.mideast']

# Download the training subset of the 20 NG dataset, with headers, footers, quotes removed
# Only keep docs from the 6 categories above
ng_train = datasets.fetch_20newsgroups(subset='train', categories=categories, 
                                      remove=('headers', 'footers', 'quotes'))

2018-12-06 19:00:27,966 : INFO : Downloading 20news dataset. This may take a few minutes.
2018-12-06 19:00:27,971 : INFO : Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


In [6]:
# Take a look at the first doc
ng_train.data[0]

'Well, the Red Sox have apparenly resigned Herm Winningham to a AAA contract.\nTed "Larry" Simmons signed him to a AAA contract then released him from\nBuffalo, allowing Lou "Curly" Gorman to circumvent the rule about not\nresigning free agents until May 1. Clearly, neither of these guys is bright\nenough to be Moe.\n\n Mike Jones | AIX High-End Development | mjones@donald.aix.kingston.ibm.com'

## Document Preprocessing
We'll need to generate a term-document matrix of word (token) counts for use in LDA.

We'll use `sklearn`'s `CountVectorizer` to generate our term-document matrix of counts. We'll make use of a few parameters to accomplish the following preprocessing of the text documents all within the `CountVectorizer`:
* `analyzer=word`: Tokenize by word
* `ngram_range=(1,2)`: Keep all 1 and 2-word grams
* `stop_words=english`: Remove all English stop words
* `token_pattern=\\b[a-z][a-z]+\\b`: Match all tokens with 2 or more (strictly) alphabet characters

In [8]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")

count_vectorizer.fit(ng_train.data)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
# Create the term-document matrix
# Transpose it so the terms are the rows
doc_word = count_vectorizer.transform(ng_train.data).transpose()

In [13]:
import pandas as pd

pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names()).head()

,0,1,2,3,4,5,6,7,8,9,...,3406,3407,3408,3409,3410,3411,3412,3413,3414,3415
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa aaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa albany,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa atlanta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa does,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
doc_word.shape

(272502, 3416)

##### Convert to gensim
We need to convert our sparse `scipy` matrix to a `gensim`-friendly object called a Corpus:

In [17]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_word)

##### Map matrix rows to words (tokens)
We need to save a mapping (dict) of row id to word (token) for later use by gensim:

In [18]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [19]:
len(id2word)

272502

## LDA
At this point we can simply plow ahead in creating an LDA model.  It requires our corpus of word counts, mapping of row ids to words, and the number of topics (3).

In [20]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=5)

2018-12-06 19:06:24,352 : INFO : using symmetric alpha at 0.3333333333333333
2018-12-06 19:06:24,354 : INFO : using symmetric eta at 0.3333333333333333
2018-12-06 19:06:24,438 : INFO : using serial LDA version on this node
2018-12-06 19:06:24,619 : INFO : running online (multi-pass) LDA training, 3 topics, 5 passes over the supplied corpus of 3416 documents, updating model once every 2000 documents, evaluating perplexity every 3416 documents, iterating 50x with a convergence threshold of 0.001000
2018-12-06 19:06:24,620 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-12-06 19:06:24,734 : INFO : PROGRESS: pass 0, at document #2000/3416
2018-12-06 19:06:26,714 : INFO : merging changes from 2000 documents into a model of 3416 documents
2018-12-06 19:06:26,809 : INFO : topic #0 (0.333): 0.002*"people" + 0.002*"don" + 0.001*"know" + 0.001*"like" + 0.001*"space" + 0.001*"just" + 0.001*"time" + 0.001*"th

2018-12-06 19:06:49,726 : INFO : topic #0 (0.333): 0.001*"image" + 0.001*"edu" + 0.001*"don" + 0.001*"people" + 0.001*"know" + 0.001*"like" + 0.001*"space" + 0.001*"just" + 0.001*"graphics" + 0.001*"time"
2018-12-06 19:06:49,731 : INFO : topic #1 (0.333): 0.002*"people" + 0.002*"armenian" + 0.001*"don" + 0.001*"said" + 0.001*"just" + 0.001*"armenians" + 0.001*"turkish" + 0.001*"like" + 0.001*"know" + 0.001*"think"
2018-12-06 19:06:49,737 : INFO : topic #2 (0.333): 0.002*"space" + 0.002*"just" + 0.002*"like" + 0.001*"don" + 0.001*"people" + 0.001*"think" + 0.001*"god" + 0.001*"know" + 0.001*"time" + 0.001*"does"
2018-12-06 19:06:49,740 : INFO : topic diff=0.246645, rho=0.386103
2018-12-06 19:06:53,035 : INFO : -11.567 per-word bound, 3033.2 perplexity estimate based on a held-out corpus of 1416 documents with 254128 words
2018-12-06 19:06:53,036 : INFO : PROGRESS: pass 4, at document #3416/3416
2018-12-06 19:06:53,718 : INFO : merging changes from 1416 documents into a model of 3416 doc

Let's take a look at what happened.  Here are the 5 most important words for each of the 3 topics we found:

In [21]:
lda.print_topics()

2018-12-06 19:06:58,964 : INFO : topic #0 (0.333): 0.002*"image" + 0.001*"jpeg" + 0.001*"edu" + 0.001*"don" + 0.001*"like" + 0.001*"space" + 0.001*"know" + 0.001*"people" + 0.001*"graphics" + 0.001*"file"
2018-12-06 19:06:58,969 : INFO : topic #1 (0.333): 0.002*"people" + 0.002*"armenian" + 0.002*"don" + 0.002*"said" + 0.001*"just" + 0.001*"like" + 0.001*"armenians" + 0.001*"think" + 0.001*"know" + 0.001*"turkish"
2018-12-06 19:06:58,977 : INFO : topic #2 (0.333): 0.002*"space" + 0.002*"like" + 0.002*"just" + 0.002*"don" + 0.001*"god" + 0.001*"people" + 0.001*"think" + 0.001*"does" + 0.001*"time" + 0.001*"know"


[(0,
  '0.002*"image" + 0.001*"jpeg" + 0.001*"edu" + 0.001*"don" + 0.001*"like" + 0.001*"space" + 0.001*"know" + 0.001*"people" + 0.001*"graphics" + 0.001*"file"'),
 (1,
  '0.002*"people" + 0.002*"armenian" + 0.002*"don" + 0.002*"said" + 0.001*"just" + 0.001*"like" + 0.001*"armenians" + 0.001*"think" + 0.001*"know" + 0.001*"turkish"'),
 (2,
  '0.002*"space" + 0.002*"like" + 0.002*"just" + 0.002*"don" + 0.001*"god" + 0.001*"people" + 0.001*"think" + 0.001*"does" + 0.001*"time" + 0.001*"know"')]

#### Topic Space
If we want to map our documents to the topic space we need to actually use the LdaModel transformer that we created above, like so:

In [22]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
lda_corpus

In [23]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

Now we can take a look at the document vectors in the topic space, which are measures of the component of each document along each topic.  Thus, at most a document vector can have num_topics=3 nonzero components in the topic space, and most have far fewer.

In [24]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:5]

[[(1, 0.33792955), (2, 0.6572214)],
 [(0, 0.010497547), (2, 0.98051757)],
 [(0, 0.011345098), (1, 0.010185722), (2, 0.9784692)],
 [(0, 0.7219733), (1, 0.1978845), (2, 0.080142245)],
 [(2, 0.99286044)]]

In [25]:
ng_train.data[0]

'Well, the Red Sox have apparenly resigned Herm Winningham to a AAA contract.\nTed "Larry" Simmons signed him to a AAA contract then released him from\nBuffalo, allowing Lou "Curly" Gorman to circumvent the rule about not\nresigning free agents until May 1. Clearly, neither of these guys is bright\nenough to be Moe.\n\n Mike Jones | AIX High-End Development | mjones@donald.aix.kingston.ibm.com'

## On your own...
- Pick a few subsets of the 20newsgroups dataset  
- Try performing LDA on this data with gensim
- Play with some of the preprocessing options and parameters for LDA, observe what happens
- See if you can use the resulting topic space to extract topic vectors
- How do your results look?
- Can you think of how you could cluster this data?